Libreries

In [4]:
run_gs=True
# define the grid search parameters and variables

n_layers=[1,2,3]
learning_rate=[0.01,0.001,0.0001,0.00001]
batch_size=[16,32,64,128]
activation_function=['relu','elu','tanh','sigmoid']
all_hyperparams=[n_layers,learning_rate,batch_size,activation_function]
population_size=6
epochs=10
sel_prt=2
rand_prt=2
generations=2

In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt # plotting library
# %matplotlib inline

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation, Dropout,BatchNormalization,Input
from tensorflow.keras.optimizers import Adam ,RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras import  backend as K
import sys
sys.path.append('./models/')
from fcunet import fcunet_model
from irnet import irnet_model
from fcmnr import fcmnr_model

from sklearn.model_selection import GridSearchCV,ParameterGrid, ParameterSampler
import random
from random import random,randrange
from operator import itemgetters
import timeit
import random
import os
import pickle
os.environ["CUDA_VISIBLE_DEVICES"]="0"
initializer = tf.keras.initializers.GlorotUniform(seed=2)
random.seed(2)

Dataset

In [ ]:
# import dataset
# from tensorflow.keras.datasets import mnist

# load dataset
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()


# count the number of unique train labels
unique, counts = np.unique(y_train, return_counts=True)
print("Train labels: ", dict(zip(unique, counts)))


# count the number of unique test labels
unique, counts = np.unique(y_test, return_counts=True)
print("\nTest labels: ", dict(zip(unique, counts)))

In [ ]:
# sample 25 mnist digits from train dataset
indexes = np.random.randint(0, x_train.shape[0], size=25)
images = x_train[indexes]
labels = y_train[indexes]


# plot the 25 mnist digits
plt.figure(figsize=(5,5))
for i in range(len(indexes)):
    plt.subplot(5, 5, i + 1)
    image = images[i]
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    
plt.show()
plt.savefig("mnist-samples.png")
plt.close('all')

Model:

In [ ]:

num_labels = len(np.unique(y_train))
# convert to one-hot vector
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

Preprocessing data

In [ ]:
# image dimensions (assumed square)
image_size = x_train.shape[1]
input_size = image_size * image_size
print(input_size)
# resize and normalize
x_train = np.reshape(x_train, [-1, input_size])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, input_size])
x_test = x_test.astype('float32') / 255

Network parameters

In [ ]:
def create_model(num_layers=2,hidden_units_start=256,activation='relu'):

    input_vec = Input(shape=input_size)
    x1=Dense(hidden_units_start,kernel_initializer=initializer)(input_vec)
    x2=Activation(activation)(x1)
    x3=Dropout(dropout,seed=2)(x2)
    for layer in range(num_layers-1):
        x4=Dense(hidden_units_start,kernel_initializer=initializer)(x3)
        x5=Activation(activation)(x4)
        x3=Dropout(dropout,seed=2)(x5)
    x6=Dense(num_labels,kernel_initializer=initializer)(x3)
    x7=Activation('softmax')(x6)

    model=Model(input_vec,x7)
    model.compile(loss='categorical_crossentropy', 
          optimizer='adam',
          metrics=['accuracy'])

    return model




In [ ]:
#INITIALIZE POPULATION

def initialize_population(population_size,num_layers,hidden_units_start,activation):

  param_grid = dict(num_layers=num_layers,hidden_units_start=hidden_units_start,activation=activation)
  grid_search_population=list(ParameterSampler(param_grid,population_size))

  potential_num_layers=[]
  potential_hidden_units_start=[]
  potential_activation=[]

  for i in range(0,population_size):
    potential_num_layers.insert(0,grid_search_population[i]['num_layers'])
    potential_hidden_units_start.insert(0,grid_search_population[i]['hidden_units_start'])
    potential_activation.insert(0,grid_search_population[i]['activation'])

  return potential_num_layers,potential_hidden_units_start,potential_activation

In [ ]:
#EVALUATE FITNESS
def evaluate_fitness(input_shape,n_layers,activation_function,learning_rate,batch_size,hp_dataset_name,weights_name,max_epochs,patience_epochs):
  #CREATE MODEL
  model=fcunet_model(n_layers,input_shape,activation_function,learning_rate) 

  start_time = timeit.default_timer()
  history = model.fit(xtrain,ytrain,
                      batch_size=batch_size,
                      epochs=max_epochs,
                      callbacks=[EarlyStopping(patience=patience_epochs)],validation_data=(xvalid,yvalid))
  end_time = timeit.default_timer()

  #EVALUATE MODEL
  prediction=model.predict(xtest)
  mae_test=mean_absolute_error(ytest,prediction)


  #SAVE THE WEIGHTS
  model.save(weights_folder+weights_name+".h5")

  #SAVE THE HYPERPARAMS AND THE METRIC
  with open(hp_dataset_name, mode='a+') as hp_dataset:
      hp_dataset_writer=csv.writer(hp_dataset,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      hp_dataset_writer.writerow([architecture_name,
                              problem_type,
                              num_features,
                              training_and_validation_samples,
                              n_layers,
                              input_shape,
                              activation_function,
                              learning_rate,
                              batch_size,
                              str(len(history.history['loss'])),
                              end_time-start_time,
                              mae_test
                              ])
  return mae_test


In [ ]:
# Evolve the hyperparameters
# Selection

def selection(evaluated_hparams,sel_prt,rand_prt,population):
    sorted_evaluated_params=sorted(evaluated_hparams,key=itemgetter('metric'),reverse=True)
    if(sel_prt+rand_prt>=len(population[0])):
      print("WARNING: Selections are bigger thant current population")
      print("WARNING: Random selection may not be taken")

    top_selection=[]
    for i in range(sel_prt):
      top_selection.insert(len(top_selection),sorted_evaluated_params[i]['hparam'])

    rand_selection=[]
    i=0
    while(i < rand_prt):
      if(len(rand_selection)+len(top_selection)>=len(population[0])):
        break

      rand_hparam=randrange(len(population[0]))
      print("Generated random {}.".format(rand_hparam))
      if(rand_hparam in top_selection or rand_hparam in rand_selection):
        continue

      rand_selection.insert(0,rand_hparam)
      i=i+1
    return top_selection,rand_selection


In [ ]:
#CROSS-OVER OPERATION

def crossover(p1,p2,population):
    child_potential_num_layers=[]
    child_potential_hidden_units_start=[]
    child_potential_activation=[]
    
    #child1
    child_potential_num_layers.insert(0,population[0][p1])
    child_potential_hidden_units_start.insert(0,population[1][p2])
    child_potential_activation.insert(0,population[2][p1])
    
    #child2
    child_potential_num_layers.insert(0,population[0][p2])
    child_potential_hidden_units_start.insert(0,population[1][p1])
    child_potential_activation.insert(0,population[2][p2])
    
    #child3
    child_potential_num_layers.insert(0,population[0][p1])
    child_potential_hidden_units_start.insert(0,population[1][p1])
    child_potential_activation.insert(0,population[2][p2])
    
    #child4
    child_potential_num_layers.insert(0,population[0][p2])
    child_potential_hidden_units_start.insert(0,population[1][p2])
    child_potential_activation.insert(0,population[2][p1])
    
    child_hparams=[child_potential_num_layers,child_potential_hidden_units_start,child_potential_activation]
    return child_hparams


In [ ]:

# MUTATION
def mutation(population,selected):
    selected_hyperparam=randrange(len(all_hyperparams))
    selected_value=randrange(len(all_hyperparams[selected_hyperparam]))
    population[selected_hyperparam][selected]=all_hyperparams[selected_hyperparam][selected_value]

    
    

In [ ]:
# OWN RANDOM GRIDSEARCH
def random_gridsearch():
    dict_all_hyperparams=dict(num_layers=num_layers,hidden_units_start=hidden_units_start,activation=activation)
    r_grid_search_population=list(ParameterSampler(dict_all_hyperparams,population_size))
    RGS_evaluated_hparams=[]
    for i in range(len(r_grid_search_population)):
            metric=evaluate_fitness(r_grid_search_population[i]['num_layers'],
                                    r_grid_search_population[i]['hidden_units_start'],
                                    r_grid_search_population[i]['activation'])
            RGS_evaluated_hparams.insert(len(RGS_evaluated_hparams),{"hparam":i,"metric":metric})
    rgs_top_hparam=sorted(RGS_evaluated_hparams,key=itemgetter('metric'),reverse=True)[0]['hparam']
    return sorted(RGS_evaluated_hparams,key=itemgetter('metric'),reverse=True)[0]['metric'],r_grid_search_population[rgs_top_hparam]


In [ ]:
# OWN GRIDSEARCH
def gridsearch():
    dict_all_hyperparams=dict(num_layers=num_layers,hidden_units_start=hidden_units_start,activation=activation)
    grid_search_population=list(ParameterGrid(dict_all_hyperparams))
    GS_evaluated_hparams=[]
    for i in range(len(grid_search_population)):
            metric=evaluate_fitness(grid_search_population[i]['num_layers'],
                                    grid_search_population[i]['hidden_units_start'],
                                    grid_search_population[i]['activation'])
            GS_evaluated_hparams.insert(len(GS_evaluated_hparams),{"hparam":i,"metric":metric})
    gs_top_hparam=sorted(GS_evaluated_hparams,key=itemgetter('metric'),reverse=True)[0]['hparam']
    return GS_evaluated_hparams,sorted(GS_evaluated_hparams,key=itemgetter('metric'),reverse=True)[0]['metric'],grid_search_population[gs_top_hparam]
    



In [ ]:
# GENETIC ALGORITHM
# MAIN
# initialize population

def genetic_algorithm_main(population_size):
    p_num_layers,p_hidden_units_start,p_activation=initialize_population(population_size=population_size,
                                                                         num_layers=num_layers,
                                                                         hidden_units_start=hidden_units_start,
                                                                         activation=activation)
    population=[p_num_layers,p_hidden_units_start,p_activation]
    print("Initial population",population)
    final_hyperparam=[]
    # evaluate hyperparams
    for generation in range(generations):
        evaluated_hparams=[]
        for i in range(population_size):
            metric=evaluate_fitness(population[0][i],population[1][i],population[2][i])
            evaluated_hparams.insert(0,{"hparam":i,"metric":metric})

        #SELECTION
        top_selection,rand_selection=selection(evaluated_hparams,sel_prt,rand_prt,population)


        # CROSS-OVER
        p1,p2=random.sample(range(0,len(top_selection)+len(rand_selection)),2)
        child_hyperparams=crossover(p1,p2,population)

        #CREATE NEW POPULATION
        new_population=[[population[0][i] for i in top_selection],
                          [population[1][i] for i in top_selection],
                          [population[2][i] for i in top_selection]]

        new_population[0]=[*new_population[0],
                                    *[population[0][i] for i in rand_selection],
                                   *child_hyperparams[0]]
        new_population[1]=[*new_population[1],
                                    *[population[1][i] for i in rand_selection],
                                   *child_hyperparams[1]]
        new_population[2]=[*new_population[2],
                                    *[population[2][i] for i in rand_selection],
                                   *child_hyperparams[2]]

        # MUTATION
        selected_to_mutate=randrange(len(top_selection)+len(rand_selection)+len(child_hyperparams[0]))
        mutation(new_population,selected_to_mutate)

        if (generation+1)==generations:
            for  hyperparam in  population:
                final_hyperparam.insert(len(population),hyperparam[top_selection[0]])

        population=new_population
        population_size=len(population[0])

    return evaluated_hparams,sorted(evaluated_hparams,key=itemgetter('metric'),reverse=True)[0]['metric'],final_hyperparam
    
                


In [ ]:
# genetic_algorithm_main
# %%capture --no-stderr
start_time = timeit.default_timer()
all_ga,top_ga, hparams_ga = genetic_algorithm_main(population_size)
stop_time = timeit.default_timer()
ga_time=stop_time - start_time

print(top_ga, hparams_ga,ga_time)



In [5]:
# random_gridsearch
# %%capture --no-stderr
start_time = timeit.default_timer()
top_rgs, hparams_rgs = random_gridsearch()
stop_time = timeit.default_timer()
rgs_time=stop_time - start_time



NameError: name 'random_gridsearch' is not defined

In [ ]:
# grid search
# %%capture --no-stderr
if(run_gs):
    start_time = timeit.default_timer()
    all_gs,top_gs, hparams_gs = gridsearch()
    stop_time = timeit.default_timer()
    gs_time=stop_time - start_time
    print(top_gs, hparams_gs,gs_time)

In [3]:
# Save and read results

if(run_gs):all_gs_sorted=sorted(all_gs,key=itemgetter('metric'),reverse=True)
all_ga_sorted=sorted(all_ga,key=itemgetter('metric'),reverse=True)
with open('results-pop'+str(population_size), 'wb') as fp:
    pickle.dump(["GA:",top_ga,hparams_ga,ga_time], fp)
    pickle.dump(["RGS:",top_rgs, hparams_rgs,rgs_time], fp)
    if(run_gs):pickle.dump(["GS:",top_gs, hparams_gs,gs_time], fp)
    if(run_gs):pickle.dump(["\n GS hparams:",all_gs_sorted],fp)
    pickle.dump(["\n GA hparams:",all_ga_sorted],fp)
with open ('results-pop'+str(population_size), 'rb') as fp:
    try:
        while True:
            print(pickle.load(fp))
    except EOFError:
        pass


NameError: name 'all_gs' is not defined